<a href="https://colab.research.google.com/github/DucAnhDang/Ima/blob/main/Bai_10_XLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi # Check GPU

Tue Dec 12 09:59:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |   1589MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install neccesssary library
## Install yolov8
!pip -q install ultralytics
## Install Detecton2 (long process so the runtime gonna take a long time [bout 5 min avg])
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
## Install Supervision
!pip -q install supervision==0.2.0


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-jcon4gm4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-jcon4gm4
  Resolved https://github.com/facebookresearch/detectron2.git to commit a0e22dbfa791e6235e4f196d5ce25e754d02be31
  Preparing metadata (setup.py) ... done


In [ ]:
# import libraries
import os
HOME = os.getcwd()
import numpy as np
import torch
import supervision as sv
from ultralytics import YOLO
#load model
model = YOLO('yolov8s.pt')

In [ ]:

vid_giaothong_pth = '/content/drive/MyDrive/traffic_camera/test_xe_2.mp4'

In [ ]:
# polygon = np.array([
#     [322, 324],
#     [490, 350],
#     [900, 610],
#     [490, 680]
# ]) # change the number for different vid (get position of abdc polygon using opencv)

# video_info = sv.VideoInfo.from_video_path(vid_giaothong_pth) # for the vid pth in the bracket
# zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# # initiate annotators
# box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
# zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=2, text_scale=1)

# # extract video frame
# generator = sv.get_video_frames_generator(vid_giaothong_pth)
# iterator = iter(generator)
# frame = next(iterator)

# # detect
# results = model(frame, imgsz=1280, conf = 0.44)[0]
# detections = sv.Detections.from_yolov8(results)
# detections = detections[(detections.class_id == 1) | (detections.class_id == 2) | (detections.class_id == 3) | (detections.class_id == 5) | (detections.class_id == 7)]
# zone.trigger(detections=detections)

# # annotate
# box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=1)
# labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
# frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
# frame = zone_annotator.annotate(scene=frame)

# %matplotlib inline
# sv.show_frame_in_notebook(frame, (16, 16))


# # the result may different because yolo often detect minivan with truck

In [ ]:
# initiate polygon zone
polygon = np.array([
    [322, 324],
    [490, 350],
    [900, 610],
    [490, 680]
]) # change the number for different vid (get position of abdc polygon using opencv)
video_info = sv.VideoInfo.from_video_path(vid_giaothong_pth)
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# initiate annotators
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=2, text_scale=1)

def process_frame(frame: np.ndarray, _) -> np.ndarray:
    # detect
    results = model(frame, imgsz=1280, conf = 0.44)[0]
    detections = sv.Detections.from_yolov8(results)
    detections = detections[(detections.class_id == 1) | (detections.class_id == 2) | (detections.class_id == 3) | (detections.class_id == 5) | (detections.class_id == 7)]
    zone.trigger(detections=detections)

    # annotate
    box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=1)
    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
    frame = zone_annotator.annotate(scene=frame)

    return frame
#This will save the vid after fully process
sv.process_video(source_path=vid_giaothong_pth, target_path=f"{HOME}/cam2-result.mp4", callback=process_frame)

from IPython import display
display.clear_output()